In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import sklearn.metrics as m
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
cars = pd.read_csv('cars.csv')
cars.columns = [c.lower() for c in cars]
cars.set_index('id', inplace=True)

print('{} rows x {} cols'.format(*cars.shape))
cars.head()

297899 rows x 8 cols


,price,year,mileage,city,state,vin,make,model
id,,,,,,,,
1,16472,2015,18681,Jefferson City,MO,KL4CJBSBXFB267643,Buick,EncoreConvenience
2,15749,2015,27592,Highland,IN,KL4CJASB5FB245057,Buick,EncoreFWD
3,16998,2015,13650,Boone,NC,KL4CJCSB0FB264921,Buick,EncoreLeather
4,15777,2015,25195,New Orleans,LA,KL4CJASB4FB217542,Buick,EncoreFWD
5,16784,2015,22800,Las Vegas,NV,KL4CJBSB3FB166881,Buick,EncoreConvenience


#### Use the cross validation techniques discussed in the lesson to figure out what kind of model works best with the cars dataset used in the lesson.

In [3]:
cars['avg_saleprice'] = cars.groupby(['year', 'make', 'model']).price.transform('mean')
cars['gt_avg'] = (cars.price > cars.avg_saleprice).astype(int)

In [4]:
cars.drop(columns=['price', 'city', 'vin', 'avg_saleprice'], inplace=True)

In [5]:
for col in ['state', 'make', 'model', 'year']:
    le = LabelEncoder().fit(cars[col])
    cars[col] = le.transform(cars[col])

In [6]:
cars.head()

,year,mileage,state,make,model,gt_avg
id,,,,,,
1,18,18681,28,7,523,0
2,18,27592,19,7,525,0
3,18,13650,32,7,526,0
4,18,25195,22,7,525,0
5,18,22800,38,7,523,0


In [7]:
X, y = cars.drop(columns='gt_avg'), cars.gt_avg

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [8]:
X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size=.3333)

In [9]:
tree = DecisionTreeClassifier(max_depth=2)

cross_val_score(tree, X_train, y_train, cv=3)

array([0.59233805, 0.59295344, 0.58173785])

In [10]:
cross_val_score(tree, X_train, y_train, cv=3, scoring='precision')

array([0.58085042, 0.58493539, 0.64582413])

### Decision Tree

In [11]:
params = {'max_depth': [2, 3, 4],
          'max_features': [None, 1, 3]}

tree = DecisionTreeClassifier()

grid = GridSearchCV(tree, params, cv=3)

grid.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [2, 3, 4], 'max_features': [None, 1, 3]})

In [12]:
results = grid.cv_results_
results.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'param_max_features', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [13]:
test_scores = results['mean_test_score']
test_scores

array([0.58900978, 0.58972727, 0.55900108, 0.63199004, 0.57583053,
       0.58546662, 0.64206638, 0.57547802, 0.60514726])

In [14]:
params = results['params']
params

[{'max_depth': 2, 'max_features': None},
 {'max_depth': 2, 'max_features': 1},
 {'max_depth': 2, 'max_features': 3},
 {'max_depth': 3, 'max_features': None},
 {'max_depth': 3, 'max_features': 1},
 {'max_depth': 3, 'max_features': 3},
 {'max_depth': 4, 'max_features': None},
 {'max_depth': 4, 'max_features': 1},
 {'max_depth': 4, 'max_features': 3}]

In [15]:
for p, s in zip(params, test_scores):
    p['score'] = s

pd.DataFrame(params).sort_values(by='score')

,max_depth,max_features,score
2,2,3.0,0.559001
7,4,1.0,0.575478
4,3,1.0,0.575831
5,3,3.0,0.585467
0,2,NaN,0.589010
1,2,1.0,0.589727
8,4,3.0,0.605147
3,3,NaN,0.631990
6,4,NaN,0.642066


In [16]:
#Decision Tree should have no max features, and 4 for max_depth

### KNN

In [ ]:
params = {'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10]}

knn = KNeighborsClassifier()

grid = GridSearchCV(knn, params, cv=3)

grid.fit(X_train, y_train)